In [1]:
!python -m pip install pandas

In [2]:
!python -m pip install openpyxl


In [3]:
import os
import re

base_path = r"G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018.CSV (survey)
    JANUARY_2018_METADATA.xlsx (metadata)
  April:
    APRIL_2018.CSV (survey)
    APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018.CSV (survey)
    OCTOBER_2018_METADATA.xlsx (metadata)
  Unmatched:
    clean_jan_2018_metadata.csv (survey)

Year 2019:
  January:
    JANUARY_2019.CSV (survey)
    JANUARY_2019_METADATA.xlsx (metadata)
  April:
    APRIL_2019_METADATA.xlsx (metadata)
    APRIL_2019.CSV (survey)
  July:
    JULY_2019_METADATA.xlsx (metadata)
    JULY_2019.CSV (survey)
  October:
    OCTOBER_2019_METADATA.xlsx (metadata)
    OCTOBER_2019.CSV (survey)

Year 2022:
  January:
    JANUARY_2022.csv (survey)
    JANUARY_2022_METADATA.xlsx (metadata)
  February:
    FEBRUARY_2022.csv (survey)
    FEBRUARY_2022_METADATA.xlsx (metadata)
  March:

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

In [5]:
import pandas as pd

# Load the second sheet of January 2018 metadata
second_sheet = load_dataset("2018", "January","metadata", 1)

# View the first few rows
second_sheet.head()


,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


In [6]:
# Load the second sheet of January 2018 metadata
second_sheet = load_dataset("2018", "January","metadata", 0)

# View the first few rows
second_sheet.head()


,QUEST,Questionnaire,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,_IDS0,(Id Items),NaN,NaN
1,NaN,NaN,NaN,NaN,PUFREG,Region
2,NaN,NaN,NaN,NaN,PUFPRV,Province
3,NaN,NaN,NaN,NaN,PUFPRRCD,Province Recode
4,NaN,NaN,NaN,NaN,PUFHHNUM,Household Unique Sequential Number


In [7]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [8]:
# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head(50)


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES
7,PUFPWGTPRV,Final Weight Based on Projection (provincial p...
8,PUFSVYMO,Survey Month
9,PUFSVYYR,Survey Year
10,PUFPSU,Psu Number
11,PUFRPL,Replicate


In [9]:
sheet2_jan2018 = load_dataset(year="2018", month="January", filetype="metadata", sheet_number=1)

print("Raw metadata sheet:")
print(sheet2_jan2018.head(20))
print(sheet2_jan2018.columns)
print(sheet2_jan2018.shape)


Raw metadata sheet:
        PUFREG_VS1               Region                            Unnamed: 2  \
0              NaN                  NaN               National Capital Region   
1              NaN                  NaN      Cordillera Administrative Region   
2              NaN                  NaN              Region I - Ilocos Region   
3              NaN                  NaN            Region II - Cagayan Valley   
4              NaN                  NaN            Region III - Central Luzon   
5              NaN                  NaN               Region IVA - CALABARZON   
6              NaN                  NaN                 Region IVB - MIMAROPA   
7              NaN                  NaN                       Region V- Bicol   
8              NaN                  NaN           Region VI - Western Visayas   
9              NaN                  NaN          Region VII - Central Visayas   
10             NaN                  NaN         Region VIII - Eastern Visayas   
11      

In [23]:
import os
import re
import pandas as pd

# ============================================================
#   Reshape function for metadata Sheet 2
# ============================================================
def process_metadata_sheet_binary1(df):
    """
    Strict horizontal reshape for metadata Sheet 2 (binary=1).
    Treats every row as data, carries forward variable/label context.
    Ensures pantay-pantay output with Code, min_val, max_val, final_val.
    """

    df = df.fillna('').astype(str).reset_index(drop=True)

    def to_int(x, default=0):
        try:
            return int(x)
        except Exception:
            return default

    def extract_ints(text):
        return [int(m.group()) for m in re.finditer(r"\d+", text)] if text else []

    def looks_like_range(text):
        s = text.lower()
        return "-" in s or "and over" in s or "to" in s

    records = []
    current_var = ""
    current_label = ""

    for _, row in df.iterrows():
        var_candidate = row.iloc[0].strip()
        if var_candidate:
            current_var = var_candidate

        label_candidate = row.iloc[1].strip()
        if label_candidate:
            current_label = label_candidate

        category = row.iloc[2].strip()
        if not category:
            continue

        # Collect numeric tokens from columns 3+
        nums = []
        for j in range(3, len(row)):
            val = row.iloc[j].strip()
            if val.isdigit():
                nums.append(int(val))

        min_val = max_val = code = final_val = 0
        cat_ints = extract_ints(category)

        if looks_like_range(category) and len(cat_ints) >= 2:
            min_val, max_val = cat_ints[0], cat_ints[1]
        elif nums:
            code = nums[0]
            final_val = code
        elif category.isdigit():
            code = int(category)
            final_val = code

        records.append({
            "Variable": current_var,
            "Label": current_label,
            "Category": category,
            "Code": code,
            "min_val": min_val,
            "max_val": max_val,
            "final_val": final_val
        })

    return pd.DataFrame.from_records(
        records,
        columns=["Variable","Label","Category","Code","min_val","max_val","final_val"]
    )


# ============================================================
#   Loader function
# ============================================================
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load survey or metadata files.
    • Survey CSV → pandas.read_csv
    • Metadata Excel → read with header=None
        - If sheet_number == 1 (Sheet 2), reshape with process_metadata_sheet_binary1
        - Else return raw DataFrame
    """

    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])

    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)

    df = pd.read_excel(file_path, sheet_name=sheet_number, header=None)

    if sheet_number == 1:
        df = process_metadata_sheet_binary1(df)

    return df


# ============================================================
#   Example calls with print output
# ============================================================

# January 2018, Sheet 2
sheet2_jan2018 = load_dataset(year="2018", month="January", filetype="metadata", sheet_number=1)
print("January 2018 (Sheet 2) preview:")
print(sheet2_jan2018.head(40))

# April 2018, Sheet 2
sheet2_apr2018 = load_dataset(year="2018", month="April", filetype="metadata", sheet_number=1)
print("\nApril 2018 (Sheet 2) preview:")
print(sheet2_apr2018.head(40))


January 2018 (Sheet 2) preview:
          Variable                Label                              Category  \
0       PUFREG_VS1               Region               National Capital Region   
1       PUFREG_VS1               Region      Cordillera Administrative Region   
2       PUFREG_VS1               Region              Region I - Ilocos Region   
3       PUFREG_VS1               Region            Region II - Cagayan Valley   
4       PUFREG_VS1               Region            Region III - Central Luzon   
5       PUFREG_VS1               Region               Region IVA - CALABARZON   
6       PUFREG_VS1               Region                 Region IVB - MIMAROPA   
7       PUFREG_VS1               Region                       Region V- Bicol   
8       PUFREG_VS1               Region           Region VI - Western Visayas   
9       PUFREG_VS1               Region          Region VII - Central Visayas   
10      PUFREG_VS1               Region         Region VIII - Eastern Visayas